# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = os.path.join(MAIN_PATH,'data')
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc

# Evaluate logits

In [5]:
DATASET = 'ImageNet'
VAL_SIZE = 0.1 #5000
SUB_VAL_SIZE = 0.2
METRIC = metrics.AURC
NUM_SPLITS = 10

In [6]:
class other_methods():
    @staticmethod
    def LDA(logits:torch.tensor, eps = 1e-20):
        logits = logits.softmax(-1)
        p1 = logits.max(-1).values.reshape(-1,1)
        sigma2 = (logits.size(-1)**2)*logits[logits!=logits.max(-1).values.reshape(-1,1)].reshape(-1,logits.size(-1)-1).var() 
        return (p1 - logits).sum(-1).pow(2) / (sigma2 + eps)
    @staticmethod
    def BK(logits:torch.tensor, alpha, beta):
        p = logits.softmax(-1).topk(2,dim=-1).values.T
        return alpha*p[0]-beta*p[1]
    @staticmethod
    def J(logits:torch.tensor):
        logits = logits.softmax(-1)
        j1 = logits.max(-1).values
        logits = logits-j1.reshape(-1,1)
        return j1 - logits.mean(-1) - logits.std(-1) 
    @staticmethod
    def ETS(logits:torch.tensor,T,w1,w2):
        return w1*logits.div(T).softmax(-1).max(-1).values + w2*logits.softmax(-1).max(-1).values
    @staticmethod
    def HTS(logits:torch.tensor,w,b):
        normalized_entropy = measures.entropy(logits).div(np.log(logits.size(-1)))
        T_h = (normalized_entropy.log()*w+b.exp()+1).log().reshape(-1,1)
        return measures.MSP(logits.div(T_h))
class optimize_other_methods():
    @staticmethod
    def ETS(logits,risk,T = None,w_range = torch.arange(0,1.01,0.01),metric = metrics.AURC, **kwargs_T):
        vals = []
        if T is None:
            T = post_hoc.optimize.T(logits, risk,**kwargs_T)
        for w1 in w_range:
            vals.append(metric(other_methods.ETS(logits,T,w1,1-w1),risk).item())
        w1 = w_range[np.argmin(vals)]
        return T,w1,1-w1
    @staticmethod
    def BK(logits:torch.tensor,risk,alpha_range = torch.arange(0,1,0.1),beta_range = torch.arange(-1,1,0.01), metric = metrics.AURC):
        vals = []
        for alpha in alpha_range:
            vals_b = []
            for beta in beta_range:
                vals_b.append(metric(other_methods.BK(logits,alpha,beta),risk).item())
            vals.append(vals_b)
            alpha,beta = np.unravel_index(np.argmin(vals),np.shape(vals))
        return alpha_range[alpha],beta_range[beta]
    @staticmethod
    def HTS(logits:torch.tensor,risk,w_range = torch.arange(-1,1,0.1),b_range = torch.arange(-3,1,0.01), metric = metrics.AURC):
        vals = []
        for w in w_range:
            vals_b = []
            for b in b_range:
                vals_b.append(metric(other_methods.HTS(logits,w=w,b=b),risk).item())
            vals.append(vals_b)
        w,b = np.unravel_index(np.argmin(vals),np.shape(vals))
        return w_range[w],b_range[b]

In [7]:
methods = {'BK': other_methods.BK,
          'ETS': other_methods.ETS,
          'HTS':other_methods.HTS}
optimizers = {'BK': optimize_other_methods.BK,
          'ETS': optimize_other_methods.ETS,
          'HTS': optimize_other_methods.HTS}

In [8]:
from collections import defaultdict
d_naurc = {m:defaultdict(list) for m in methods.keys()}
d_baseline = defaultdict(list)
acc = defaultdict(list)

In [9]:
seed = SEED
for i in range(NUM_SPLITS):
    print(i+1)
    for model_arc in models.list_models():
        print(model_arc)
        with torch.no_grad():
            logits_val,labels_val,logits_test,labels_test = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                split = 'test', device = dev),VAL_SIZE,seed = seed)
            logits_val,labels_val = logits_val[:int(SUB_VAL_SIZE*labels_val.size(0))],labels_val[:int(SUB_VAL_SIZE*labels_val.size(0))]
            risk_val = measures.wrong_class(logits_val,labels_val).float()
            risk_test = measures.wrong_class(logits_test,labels_test).float()
        acc[model_arc].append((1-risk_test.mean().item()))

        for method,fn in methods.items():
            params = optimizers[method](logits_val,risk_val)
            d_naurc[method][model_arc].append(metrics.N_AURC(fn(logits_test,*params),risk_test))

        d_baseline[model_arc].append(metrics.N_AURC(measures.MSP(logits_test),risk_test))
    seed += 10
models_list = list(acc.keys())

1
alexnet
convnext_base
convnext_large
convnext_small
convnext_tiny
densenet121
densenet161
densenet169
densenet201
efficientnet_b0
efficientnet_b1
efficientnet_b2
efficientnet_b3
efficientnet_b4
efficientnet_b5
efficientnet_b6
efficientnet_b7
efficientnet_v2_l
efficientnet_v2_m
efficientnet_v2_s
googlenet
inception_v3
maxvit_t
mnasnet0_5
mnasnet0_75
mnasnet1_0
mnasnet1_3
mobilenet_v2
mobilenet_v3_large
mobilenet_v3_small
regnet_x_16gf
regnet_x_1_6gf
regnet_x_32gf
regnet_x_3_2gf
regnet_x_400mf
regnet_x_800mf
regnet_x_8gf
regnet_y_128gf
regnet_y_16gf
regnet_y_1_6gf
regnet_y_32gf
regnet_y_3_2gf
regnet_y_400mf
regnet_y_800mf
regnet_y_8gf
resnet101
resnet152
resnet18
resnet34
resnet50
resnext101_32x8d
resnext101_64x4d
resnext50_32x4d
shufflenet_v2_x0_5
shufflenet_v2_x1_0
shufflenet_v2_x1_5
shufflenet_v2_x2_0
squeezenet1_0
squeezenet1_1
swin_b
swin_s
swin_t
swin_v2_b
swin_v2_s
swin_v2_t
vgg11
vgg11_bn
vgg13
vgg13_bn
vgg16
vgg16_bn
vgg19
vgg19_bn
vit_b_16
vit_b_32
vit_h_14
vit_l_16
vit_l_32


In [10]:
baseline = np.array(list(d_baseline.values()))

In [11]:
for method in methods:
    d_t = d_naurc[method]
    string = f'{method}'
    string += f" & {post_hoc.significant(baseline - np.array(list(d_t.values())),0.01).mean():.5f}"+r' {\footnotesize $\pm$'+f"{post_hoc.significant(baseline - np.array(list(d_t.values())),0.01).mean(0).std():.5f}"+"}"
    print(string + r' \\')

BK & 0.03795 {\footnotesize $\pm$0.00067} \\
ETS & 0.05569 {\footnotesize $\pm$0.00165} \\
HTS & 0.05927 {\footnotesize $\pm$0.00280} \\
